# Some KEY POINTS

- For CRSP stock data, price and share outstanding need to be adjusted.

-

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import time as time
import warnings
import matplotlib.pyplot as plt
from pandas.tseries.offsets import MonthEnd
import pyreadstat as pyreadstat

warnings.filterwarnings('ignore')

```SAS
libname tfn "/wrds/tfn/sasdata/s12";
libname mfl "/wrds/mfl/sasdata";
libname ff  "/wrds/ff/sasdata";

/* Step 1.1. Specifying Options */
%let begdate = 01MAR1980;
%let enddate = 31DEC2008;
%let mfldate = 01JAN2012;
```

In [2]:
begdate = '1980-03-01'
enddate = '2008-12-31'
mfldate = '2012-01-01'

```SAS
/* Step 1.2. Get CRSP Prices, Returns, and Adjustment Factors from MSF Dataset */
/* and align CRSP month-end dates */ 
data Price;
set crsp.msf (keep=permno date cfacpr cfacshr shrout prc ret);
by permno date;
where date between "&begdate"d and "&enddate"d;
DATE = INTNX('MONTH',date,0,'E');
TSO=shrout*cfacshr*1000;
P  = abs(prc)/cfacpr;
P_1= lag(P); if first.permno then P_1=.;
label TSO = "Total Shares Outstanding, Adjusted";
label P = "Price at t Period End, Adjusted";
label P_1 = "Price at (t-1), Adjusted";
if TSO>0; format TSO comma12.0 ret percentn8.2 prc: dollar12.3;
drop prc cfacpr shrout;
run; 
```

I translate the above SAS code into Python code as follows.

In [3]:
# Read teh CRSP msf data files and extract the columns
# - permno: CRSP Permanent Security Identifier
# - cfacpr: Cumulative Factor to Adjust Prices
# - cfacshr: Cumulative Factor to Adjust Shares/Vol
# - shrout: Shares Outstanding
# - prc: Closing Price or Negative Bid/Ask Average
# - ret: Holding Period Return
crsp_msf,meta    = pyreadstat.read_sas7bdat('crsp_msf.sas7bdat',
                                            usecols = ['DATE','PERMNO','CFACPR',
                                                       'CFACSHR','SHROUT','PRC','RET'])
# Transform the date float into datetime objects
crsp_msf['DATE'] = pd.to_timedelta(crsp_msf['DATE'], unit='D') + pd.Timestamp(1960, 1, 1)

In [4]:
# Rename the columns' names for following studies
# Extract the data in a given time span
crsp_msf         = crsp_msf[['DATE','PERMNO','CFACPR','CFACSHR','SHROUT','PRC','RET']]
crsp_msf.columns = ['date','permno','cfacpr','cfacshr','shrout','prc','ret']
crsp_msf         = crsp_msf[crsp_msf['date'] <= enddate]
crsp_msf         = crsp_msf[crsp_msf['date'] >= begdate]

In [5]:
# Use the datetime object to change the date into the end of every month
# that is align CRSP month-end dates
crsp_msf['date'] = pd.to_datetime(crsp_msf['date']) + MonthEnd(0)
Price            = crsp_msf.copy()
del crsp_msf
Price.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2527187 entries, 0 to 4592630
Data columns (total 7 columns):
 #   Column   Dtype         
---  ------   -----         
 0   date     datetime64[ns]
 1   permno   float64       
 2   cfacpr   float64       
 3   cfacshr  float64       
 4   shrout   float64       
 5   prc      float64       
 6   ret      float64       
dtypes: datetime64[ns](1), float64(6)
memory usage: 154.2 MB


In [6]:
# Construct the Price data set from CRSP.msf databased

# TSO = "Total Shares Outstanding, Adjusted";
Price['TSO'] = Price['shrout'] * Price['cfacshr'] * 1000

# P = "Price at t Period End, Adjusted";
Price['P']   = Price['prc'].abs()/Price['cfacpr']

# P_1 = "Price at (t-1), Adjusted"; 
# Backward shift the price for every stock 
Price['P_1'] = Price.groupby('permno')['P'].shift(1)

# Scale the return by 100
Price['ret'] = Price['ret'] * 100

# After the adjustment for price and share outstanding, delete the useless columns
Price.drop(['prc','cfacpr','shrout'],axis = 1,inplace = True)

# Keep those stock with positive 'total share outstanding'
Price            = Price[Price['TSO'] > 0]
# Change the 'total share outstanding' and 'permno' as the integer
Price            = Price.reset_index(drop = True)
Price['permno']  = Price['permno'].astype(int)
# Price['permno']  = Price['permno'].astype(str)
Price['TSO']     = Price['TSO'].astype(int)

In [7]:
Price.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2490587 entries, 0 to 2490586
Data columns (total 7 columns):
 #   Column   Dtype         
---  ------   -----         
 0   date     datetime64[ns]
 1   permno   int64         
 2   cfacshr  float64       
 3   ret      float64       
 4   TSO      int64         
 5   P        float64       
 6   P_1      float64       
dtypes: datetime64[ns](1), float64(4), int64(2)
memory usage: 133.0 MB


```SAS
/* Step 1.3. Get report and vintage dates from Thomson-Reuters Mutual Fund Holdings */
/* Exclude Non-Equity Funds from Holdings data that is reported as of Fiscal Quarter End */
/* First, Keep First Vintage with Holdings Data for Each RDATE-FUNDNO */
proc sql;
create table First_Vint
as select distinct intnx("month",rdate,0,"E") as rdate format date9., fdate, fundno
from tfn.s12type1
where ("&begdate"d <= rdate <="&enddate"d and ioc not in (1,5,6,7))
group by fundno, intnx("month",rdate,0,"E")
having fdate=min(fdate) and max(rdate)=rdate
order by fundno, rdate desc; 
quit; 
```

In [8]:
# Get report and vintage dates from Thomson-Reuters Mutual Fund Holdings
First_Vint,meta      = pyreadstat.read_sas7bdat('tfn_s12type1.sas7bdat',
                                                usecols = ['fundno','rdate','fdate','ioc'])
First_Vint['rdate']  = pd.to_datetime(First_Vint['rdate'])
First_Vint['fdate']  = pd.to_datetime(First_Vint['fdate'])

In [9]:
# Extract the dataset with following columns
# - fundno: Fund Number
# - rdate: Report Date
# - fdate: File Date
# - ioc: Investment Objective Code
First_Vint = First_Vint[['fundno','rdate','fdate','ioc']]
First_Vint = First_Vint[First_Vint['rdate'] >= begdate]
First_Vint = First_Vint[First_Vint['rdate'] <= enddate]

In [10]:
# Exclude Non-Equity Funds from Holdings data that is reported as of Fiscal Quarter End
# According to the 'investment object code'
# 
First_Vint = First_Vint[~First_Vint['ioc'].isin([1,5,6,7])]

In [11]:
# Align 'rdate' and 'fdate' as the month-end dates
First_Vint['rdate']  = pd.to_datetime(First_Vint['rdate']) + MonthEnd(0)
First_Vint['fdate']  = pd.to_datetime(First_Vint['fdate']) + MonthEnd(0)
First_Vint['fundno'] = First_Vint['fundno'].astype(int)
# First_Vint['fundno'] = First_Vint['fundno'].astype(str)
# First_Vint['fundno'] = First_Vint['fundno'].str.zfill(6)
First_Vint.drop('ioc',axis = 1,inplace = True)

In [12]:
# Extract the month for every report date and store
# the results in a new column with name 'rdateM'
TmpSeries            = pd.DataFrame(First_Vint['rdate'])
TmpSeries.set_index('rdate',inplace = True)
TmpSeries            = TmpSeries.to_period('M').reset_index()
TmpSeries.index      = First_Vint['rdate'].index
First_Vint['rdateM'] = TmpSeries
del TmpSeries

In [13]:
# In order to eliminate the dumpliates, here we examine for every fund and its every month
# choose the minimum of 'fdate' as the exact 'fdate'
#        the maximum of 'rdate' as the exact 'rdate'
# Group the First_Vint according to 'fundno' and 'rateM'
# and for every category, we extract the minimum of 'fdate' and 
# maximum of 'rdate' 
Groupby02 = First_Vint.groupby(['fundno','rdateM']).agg({'fdate': 'min',
                                                       'rdate': 'max'})
# Since given the 'rdate', the 'fdate' may still be duplicated, here we group the 
# the data according to 'fundno' and 'rdate' and choose the minium of 'fdate' as
# the exact 'fdate'
FirstVint = Groupby02.groupby(['fundno','rdate']).agg({'fdate':'min'})
del Groupby02

In [14]:
FirstVint.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 488585 entries, (np.int64(5), Timestamp('1994-10-31 00:00:00')) to (np.int64(99000), Timestamp('1990-09-30 00:00:00'))
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   fdate   488585 non-null  datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 6.9 MB


```SAS

/* Until 2003, funds were only required to disclose their holdings semi-annually  */
/* Approximately 60% of funds additionally report quarterly holdings              */
/* Use 6-Months as cut-off for portfolio holding period                           */
data First_Vint; set First_Vint;
by fundno descending rdate;	format nrdate date9.;
nrdate = lag(rdate); if first.fundno then nrdate = intnx("month",rdate,6,"E");
nrdate = min(nrdate,intnx("month",rdate,6,"E"));
run; 

proc sort data=First_Vint nodupkey; by fundno fdate; run; 
```

In [15]:
# Construct a new column 'nrdate' showing the next required reported date
FirstVint02           = FirstVint.sort_index(ascending=[True,False]).reset_index()
FirstVint02['nrdate'] = FirstVint02.groupby('fundno')['rdate'].shift(1).reset_index(drop = True)

# For the last 'nrdate' of every fund, we assign that it is the last 'rdate'
# plus 6 months
NaNindex              = FirstVint02.index[FirstVint02['nrdate'].isna()]
FirstVint02['nrdate'][NaNindex] = FirstVint02['nrdate'][NaNindex+1] + MonthEnd(6)
del NaNindex

In [16]:
# Since some funds may report after a longer period than 6 months, 
# for these cases, we only choose the closest one to the current 'rdate'
FirstVint02['nrdate'] = pd.concat([FirstVint02['nrdate'],FirstVint02['rdate'] + MonthEnd(6)],
                                  axis = 1).min(axis = 1)
# Remove the duplications and keep the first iterm
FirstVint02.drop_duplicates(subset = ['fundno','fdate'],inplace = True)
FirstVint02.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 488585 entries, 0 to 488584
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   fundno  488585 non-null  int64         
 1   rdate   488585 non-null  datetime64[ns]
 2   fdate   488585 non-null  datetime64[ns]
 3   nrdate  488585 non-null  datetime64[ns]
dtypes: datetime64[ns](3), int64(1)
memory usage: 14.9 MB


MFLINKS provides links between the 98% of **the domestic equity funds in CRSP** and
**Thomson-Reuters**. The database consists of **two tables** that provide **the mapping between
CRSP and Thomson-Reuters mutual fund identifiers, to a common Wharton Financial
Institution Center Number (WFICN)**, which is a unique and permanent fund portfolio
identifier. 

One should use the **'fundno'** and **'fdate'** to match.

```SAS
/* Step 1.4. Add WFICN portfolio identifiers from MFLINKS */
proc sql;
create table First_Vint
as select b.wficn as wficn_old, a.*
from First_Vint as a left join  mfl.mflink2 as b
on  a.fundno=b.fundno and a.fdate = b.fdate;
quit; 
```

In [17]:
# Add WFICN portfolio identifiers from MFLINKS
mfl_mflink2,meta     = pyreadstat.read_sas7bdat('mfl_mflink2.sas7bdat',
                                                usecols = ['fundno','wficn','fdate'])

In [18]:
mfl_mflink2['fdate'] = pd.to_datetime(mfl_mflink2['fdate'])

In [19]:
# Extract the columns 'fundno', 'wficn' and 'fdate'
mfl_mflink2['fundno'] = mfl_mflink2['fundno'].astype(int)
mfl_mflink2           = mfl_mflink2.rename(columns = {'wficn':'wficn_old'})
# Merge TFN data with the mlf_link datasets for the following merge
FirstVint03           = pd.merge(FirstVint02,mfl_mflink2,on = ['fundno','fdate'])

In [20]:
FirstVint03.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 488585 entries, 0 to 488584
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   fundno     488585 non-null  int64         
 1   rdate      488585 non-null  datetime64[ns]
 2   fdate      488585 non-null  datetime64[ns]
 3   nrdate     488585 non-null  datetime64[ns]
 4   wficn_old  164888 non-null  float64       
dtypes: datetime64[ns](3), float64(1), int64(1)
memory usage: 18.6 MB


```SAS
data First_Vint; 
length WFICN 6.;
set First_Vint;
by fundno fdate; retain wficn;
if first.fundno or not missing(wficn_old) then wficn=wficn_old;
if missing(wficn_old) and fdate<"&mfldate"d then wficn=.;
drop wficn_old;
run;

proc sort data=First_Vint nodupkey; by wficn rdate; run; 

```

In [21]:
# 首先转换日期格式
mfldate_dt = pd.to_datetime(mfldate)

# Transform the 'wficn' into integer 
FirstVint03['wficn_old'] = FirstVint03['wficn_old'].astype('Int64')
FirstVint03.sort_values(['fundno','fdate'],inplace = True)
FirstVint03['wficn']     = FirstVint03['wficn_old'].copy()
# SAS 逻辑: if missing(wficn_old) and fdate < mfldate then wficn = .
FirstVint03.loc[(FirstVint03['wficn_old'].isna()) & 
                (FirstVint03['fdate'] < mfldate_dt), 'wficn'] = np.nan

In [22]:
FirstVint03 = FirstVint03.drop_duplicates(subset = ['wficn','rdate'])
FirstVint03 = FirstVint03.sort_values(['wficn','rdate'])

In [23]:
FirstVint03.info()

<class 'pandas.core.frame.DataFrame'>
Index: 164865 entries, 1976 to 64
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   fundno     164865 non-null  int64         
 1   rdate      164865 non-null  datetime64[ns]
 2   fdate      164865 non-null  datetime64[ns]
 3   nrdate     164865 non-null  datetime64[ns]
 4   wficn_old  164531 non-null  Int64         
 5   wficn      164531 non-null  Int64         
dtypes: Int64(2), datetime64[ns](3), int64(1)
memory usage: 9.1 MB


In [24]:
del FirstVint02
del FirstVint
del mfl_mflink2

**Thomson Financial s12type1 file** reports two dates: RDATE and FDATE. 
As of this writing, out of 1133304 observations in that file, only 242739 (21%) had the two dates equal. 
- RDATE is the date as of which **the positions are held by the fund**. 
- FDATE is a 'vintage date' and is used as a **key** for joining some databases. 

Not sure whybit is called 'F'DATE, but it has nothing to do with a filing date for our intents and purposes. Check out http://goo.gl/lMI3l (esp. page 5) for details (you have to be a WRDS member).

one last thing to do is to **adjust shares held by the fund for stock splits**, etc. 

- The number of shares is reported as of FDATE but the effective holding date is RDATE. If a particular stock (PERMNO) had some event such as a stock split which affected the number of shares outstanding, the reported number of shares needs to be adjusted.

As an example, let's continue with fundno = 1099 (ROYCE HERITAGE FUND) reference above. Its
wficn is 105794, and its SEC's Edgar filings are here: http://goo.gl/t5mAi. On RDATE=March 31 2010, the fund held permno = 76489 (LUFKIN INDUSTRIES), and the number of shares reported in s12type3 file is 43,400. However, if we dig up the actual SEC filing for that quarter (http://goo.gl/yTkHb), we'll see that the fund actually held only 21,700 shares which amounted to an investment of $1,717,555. 

Why the difference? It's because Lufkin Industries had a 2:1 split in June 2010, and the number of shares in s12type3 is already adjusted for the split. So we need to re-adjust it back before we compute dollar investments for every fund-month-stock observation.

```SAS
/* Step 1.5. Extract Holdings by Merging TR-MF S12type1 and S12type3 Sets */
proc sql;
create view Holdings  /* Add Holdings Data */
  as select a.rdate,a.nrdate,a.fdate,a.wficn,a.fundno,b.cusip,b.shares
  from First_Vint as a, tfn.s12type3 as b
  where a.fdate=b.fdate and a.fundno=b.fundno; 
 ```

In [25]:
tfn_s12type3, meta   = pyreadstat.read_sas7bdat('tfn_s12type3.sas7bdat', 
                                                usecols = ['fundno','fdate','cusip','shares'])

In [26]:
tfn_s12type3['fdate'] = pd.to_datetime(tfn_s12type3['fdate'])

In [27]:
Holdings = pd.merge(FirstVint03,tfn_s12type3,on = ['fundno','fdate'])

In [28]:
# The Holdings include the 
# - fundno
# - rdate:  the exact data date, for analysis
# - fdate:  the file date, a key for merge
# - wficn:  the WRDS identifier from MFLINK
# - cusip:  the holding of stock names
# - shares: the share of hold stock, not adjusted yet
Holdings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18696217 entries, 0 to 18696216
Data columns (total 8 columns):
 #   Column     Dtype         
---  ------     -----         
 0   fundno     int64         
 1   rdate      datetime64[ns]
 2   fdate      datetime64[ns]
 3   nrdate     datetime64[ns]
 4   wficn_old  Int64         
 5   wficn      Int64         
 6   cusip      object        
 7   shares     float64       
dtypes: Int64(2), datetime64[ns](3), float64(1), int64(1), object(1)
memory usage: 1.1+ GB


In [29]:
del tfn_s12type3

```SAS 
create view Holdings1 /* Map Holdings CUSIP to CRSP Unique Identifier PERMNO */
  as select a.rdate,a.nrdate,a.fdate,a.wficn,a.fundno,b.permno,a.shares
  from Holdings as a, (select distinct ncusip, permno from crsp.msenames
                       where not missing(ncusip)) as b
  where a.cusip=b.ncusip; 
```

In [30]:
crsp_msenames,meta    = pyreadstat.read_sas7bdat('crsp_msenames.sas7bdat',
                                                  usecols = ['NCUSIP', 'PERMNO'])
crsp_msenames.columns = ['permno','ncusip']
crsp_msenames         = crsp_msenames.drop_duplicates(subset = ['ncusip', 'permno'])

In [31]:
Holdings1             = pd.merge(Holdings,crsp_msenames,left_on= 'cusip', right_on = 'ncusip')
del crsp_msenames

In [32]:
Holdings1['permno'] = Holdings1['permno'].astype('Int64')
Holdings1['shares'] = Holdings1['shares'].astype('Int64')
Holdings1.drop(['ncusip','cusip'],inplace = True, axis = 1)
Holdings1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18455926 entries, 0 to 18455925
Data columns (total 8 columns):
 #   Column     Dtype         
---  ------     -----         
 0   fundno     int64         
 1   rdate      datetime64[ns]
 2   fdate      datetime64[ns]
 3   nrdate     datetime64[ns]
 4   wficn_old  Int64         
 5   wficn      Int64         
 6   shares     Int64         
 7   permno     Int64         
dtypes: Int64(4), datetime64[ns](3), int64(1)
memory usage: 1.2 GB




```SAS
create table Holdings2 /* Adjust Shares on Vintage Dates */
  as select a.rdate, a.nrdate, a.wficn, a.fundno, a.permno, 
      a.shares*b.cfacshr as shares_adj label = "Adjusted Shares Held"
  from Holdings1 as a, price as b
  where a.permno=b.permno and a.fdate=b.date;
quit;
```

In [33]:
# Due to the stock split, the shares of the holdings of funds need to be adjusted 
# on 'Vintage Dates', the 'fdate'.
Tmp               = pd.merge(Holdings1,Price,left_on = ['permno','fdate'],
                             right_on = ['permno','date'])
Tmp['shares_adj'] = Tmp['shares']*Tmp['cfacshr']
Holdings2         = Tmp[['rdate','nrdate','wficn','fundno','permno','shares_adj']]
del Tmp

In [34]:
Holdings2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18292061 entries, 0 to 18292060
Data columns (total 6 columns):
 #   Column      Dtype         
---  ------      -----         
 0   rdate       datetime64[ns]
 1   nrdate      datetime64[ns]
 2   wficn       Int64         
 3   fundno      int64         
 4   permno      Int64         
 5   shares_adj  Float64       
dtypes: Float64(1), Int64(2), datetime64[ns](2), int64(1)
memory usage: 889.7 MB


```SAS
/* Step 1.6. Add CRSP Market Data to Holdings: Populate Returns */
/* between 2 holding reports. Use holding dollar value at (t-1) as */
/* weights for buy & hold portfolio returns */
proc sql;
Create table MF_Holdings 
as select b.date, a.rdate, a.wficn, a.fundno, a.permno, a.shares_adj, b.ret,
  (a.shares_adj*b.P_1) as HVALUE_1 label="MF Holding $ Value at (t-1)"
from Holdings2 as a, price as b
where a.permno=b.permno and a.rdate < b.date <= a.nrdate;
quit; 

/* Sanity Checks for Duplicates - Ultimately, Should be 0 Duplicates */
/* If No Errors, then Duplicates can be due to 2 historical CUSIPs */
/*    (Separate Holdings by Same Manager) mapping to the same permno */
proc sort data=MF_Holdings out=MF_Holdings nodupkey;
by wficn date rdate permno; where HVALUE_1>0;
run; 
```

In [35]:
# In order to avoid the memory exploding, I have to seperate the merge
# according to stock identifier one-by-one.
# Based on the above Thomas Financial data operation, we can obtain the 
# holdings of every mutual fund at every date. 
# In order to examine the details of holdings, we would like to combine the 
# Price information from the CRSP based on the 'permno'

#  Add CRSP Market Data to Holdings: Populate Returns 
#  between 2 holding reports. 
ValsList    = ['fundno','wficn','date', 'rdate','permno', 'shares_adj','ret','P_1']
permnoArray = Holdings2['permno'].unique()
PERMNO      = permnoArray[0]
Part01      = Holdings2[Holdings2['permno'] == PERMNO]
Part02      = Price[Price['permno'] == PERMNO]
Part03      = pd.merge(Part01,Part02,on = 'permno')
Part03      = Part03[(Part03['rdate'] < Part03['date']) & 
                     (Part03['date'] <= Part03['nrdate'])][ValsList]
MF_Holdings = []
MF_Holdings.append(Part03.reset_index(drop = True))
for i in range(1,len(permnoArray)): 
    if i%1000 == 0:
        print(i)
    PERMNO  = permnoArray[i]
    Part01  = Holdings2[Holdings2['permno'] == PERMNO]
    Part02  = Price[Price['permno'] == PERMNO]
    Part03  = pd.merge(Part01,Part02,on = 'permno')
    Part03  = Part03[(Part03['rdate'] < Part03['date']) & 
                     (Part03['date'] <= Part03['nrdate'])][ValsList]
    MF_Holdings.append(Part03)

MF_Holdings = pd.concat(MF_Holdings).reset_index(drop = True)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000


In [36]:
# Use holding dollar value at (t-1) as weights for buy & hold portfolio returns 
MF_Holdings['HVALUE_1'] = MF_Holdings['shares_adj'] * MF_Holdings['P_1']
MF_Holdings             = MF_Holdings.sort_values(['wficn','date','rdate','permno'])
MF_Holdings             = MF_Holdings.drop_duplicates(subset = ['wficn','date','rdate','permno'],
                                                      keep = 'first')
MF_Holdings             = MF_Holdings.reset_index(drop = True)
MF_Holdings             = MF_Holdings[MF_Holdings['HVALUE_1'] > 0]
# MF_Holdings.to_csv('MF_Holdings.csv')

In [37]:
MF_Holdings.info()

<class 'pandas.core.frame.DataFrame'>
Index: 65969856 entries, 0 to 66034333
Data columns (total 9 columns):
 #   Column      Dtype         
---  ------      -----         
 0   fundno      int64         
 1   wficn       Int64         
 2   date        datetime64[ns]
 3   rdate       datetime64[ns]
 4   permno      Int64         
 5   shares_adj  Float64       
 6   ret         float64       
 7   P_1         float64       
 8   HVALUE_1    Float64       
dtypes: Float64(2), Int64(2), datetime64[ns](2), float64(2), int64(1)
memory usage: 5.2 GB


```SAS
/* Step 1.7. Calculate Assets and Holdings Returns, at the Portfolio Level */
proc means data=MF_Holdings noprint;
by wficn date;
weight HVALUE_1;
var ret;
output out=MF_PortRets (drop=_TYPE_ _FREQ_) N=NSTOCKS MEAN=HRET SUMWGT=Assets_1;
run; 
```

In [39]:
# Extract the columns 'wficn','date','ret','HVALUE_1' from the MF_Holdings
MF_PortRets = MF_Holdings[['wficn','date','ret','HVALUE_1']].copy()

# Assign the NaN terms in 'wficn' as 0
MF_PortRets['wficn'][MF_PortRets['wficn'].isna()] = 0

# here the 'hret' means the return when the fund holds the same portfolio
# at last period, that is, the fund will keep the potfolio composition 
# unchanged, that is the values weights of every stock are assumed to be
# the same.
MF_PortRets['hret'] = MF_PortRets['ret'] * MF_PortRets['HVALUE_1']
TmpNstock           = MF_PortRets.groupby(['wficn','date'])['HVALUE_1'].count().reset_index()
MF_PortRets         = MF_PortRets.groupby(['wficn','date'])[['hret','HVALUE_1']].sum()
MF_PortRets['hret'] = MF_PortRets['hret']/MF_PortRets['HVALUE_1']

# Assign total values of last period as 'Assset_1'
MF_PortRets['Assets_1'] = MF_PortRets['HVALUE_1']
# Assign the total number of stocks invested as 'Nstocks'
MF_PortRets['Nstocks']  = TmpNstock['HVALUE_1']
del TmpNstock
MF_PortRets.drop('HVALUE_1',axis = 1,inplace = True)

In [41]:
MF_PortRets.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 560658 entries, (np.int64(0), Timestamp('1980-04-30 00:00:00')) to (np.int64(601802), Timestamp('2008-11-30 00:00:00'))
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   hret      560658 non-null  Float64
 1   Assets_1  560658 non-null  Float64
 2   Nstocks   0 non-null       Int64  
dtypes: Float64(2), Int64(1)
memory usage: 16.6 MB


```SAS
/* Exclude Funds that hold fewer than 10 stocks and manage less then $5 mil */

data MF_PortRets; set MF_PortRets; format HRET percentn8.4;
Assets_1=Assets_1/1000000; Assets = Assets_1*(1+HRET);
if Assets>=5 and Nstocks>=10;
label HRET = 'Value-Weighted Monthly Portfolio Holdings Return';
label Assets = "Mutual Fund Portfolio Assets at Month t, $Million";
label Assets_1= "Mutual Fund Portfolio Assets at Month (t-1), $Million";
label Nstocks = "Number of Stocks Held as of Month t";
run; 
```

In [42]:
MF_PortRets['Assets_1']  = MF_PortRets['Assets_1']/1000000
MF_PortRets['Assets']    = MF_PortRets['Assets_1'] * (1 + MF_PortRets['hret']/100)

In [43]:
# MF_PortRets includes
# HRET     = 'Value-Weighted Monthly Portfolio Holdings Return';
# Assets   = "Mutual Fund Portfolio Assets at Month t, $Million";
# Assets_1 = "Mutual Fund Portfolio Assets at Month (t-1), $Million";
# Nstocks  = "Number of Stocks Held as of Month t";
MF_PortRets              = MF_PortRets[(MF_PortRets['Assets'] >= 5) & 
                                       (MF_PortRets['Nstocks'] >= 10)]

In [44]:
MF_PortRets.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 0 entries
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   hret      0 non-null      Float64
 1   Assets_1  0 non-null      Float64
 2   Nstocks   0 non-null      Int64  
 3   Assets    0 non-null      Float64
dtypes: Float64(3), Int64(1)
memory usage: 60.6 KB


```SAS
/* Step 2.1. Extract CRSP Mutual Funds Performance and Characteristics */
/* Merge fraction of portfolio in equity assets with the objective code data */
proc sql; 
create table Fund_Style
as select a.crsp_fundno, a.si_obj_cd, a.wbrger_obj_cd, a.policy, a.lipper_class, b.avrcs
from crsp.fund_style a left join (select distinct crsp_fundno, sum(per_com)/count(per_com) as avrcs
from crsp.fund_summary group by crsp_fundno) b
on a.crsp_fundno=b.crsp_fundno;
quit; 
```

In [45]:
# Extract the columns from the dataset 'crsp.fund_summary'
# including:
#  - crsp_fundno: Fund Identifier
#  - per_com:     Amount of fund invested in Common Stocks, the percentage of fund invested
#  - avrcs:       the averaged of the amount recorded for every fund
crsp_fund_summary,meta     = pyreadstat.read_sas7bdat('crsp_fund_summary.sas7bdat',
                                                     usecols = ['crsp_fundno', 'per_com'])
Tmp                        = crsp_fund_summary.groupby('crsp_fundno')['per_com']
crsp_fund_summary          = crsp_fund_summary.drop_duplicates('crsp_fundno')
crsp_fund_summary.set_index('crsp_fundno',inplace = True)
crsp_fund_summary['avrcs'] = Tmp.sum()/Tmp.count()
crsp_fund_summary.reset_index(inplace = True)

In [46]:
crsp_fund_summary.drop('per_com',inplace = True,axis = 1)
crsp_fund_summary['crsp_fundno'] = crsp_fund_summary['crsp_fundno'].astype(int)
crsp_fund_summary['crsp_fundno'] = crsp_fund_summary['crsp_fundno'].astype(str)
crsp_fund_summary['crsp_fundno'] = crsp_fund_summary['crsp_fundno'].str.zfill(6)

In [47]:
# Extract the columns from the dataset 'crsp.fund_style'
# including:
#  - crsp_fundno:    Fund Identifier
#  - si_obj_cd:      Strategic Insight Objective Code
#  - wbrger_obj_cd:  Wiesenberger Fund Type Code, Identifying Fund Strategy
#  - policy:         Type of Securities Mainly Held by Fund
#  - lipper_class:   Lipper Classification Code

crsp_fund_style,meta             = pyreadstat.read_sas7bdat('crsp_fund_style.sas7bdat',
                                                            usecols = ['crsp_fundno', 'si_obj_cd', 
                                                                       'wbrger_obj_cd', 'policy',
                                                                       'lipper_class'])
crsp_fund_style['crsp_fundno']   = crsp_fund_style['crsp_fundno'].astype(int)
crsp_fund_style['crsp_fundno']   = crsp_fund_style['crsp_fundno'].astype(str)
crsp_fund_style['crsp_fundno']   = crsp_fund_style['crsp_fundno'].str.zfill(6)
crsp_fund_style.set_index('crsp_fundno',inplace = True)

In [48]:
crsp_fund_style.info()

<class 'pandas.core.frame.DataFrame'>
Index: 178842 entries, 000001 to 096711
Data columns (total 4 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   si_obj_cd      178842 non-null  object
 1   wbrger_obj_cd  178842 non-null  object
 2   policy         178842 non-null  object
 3   lipper_class   178842 non-null  object
dtypes: object(4)
memory usage: 6.8+ MB


In [49]:
fund_style = crsp_fund_style.join(crsp_fund_summary.set_index('crsp_fundno'),
                                  on = 'crsp_fundno').reset_index()

In [50]:
fund_style.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178842 entries, 0 to 178841
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   crsp_fundno    178842 non-null  object 
 1   si_obj_cd      178842 non-null  object 
 2   wbrger_obj_cd  178842 non-null  object 
 3   policy         178842 non-null  object 
 4   lipper_class   178842 non-null  object 
 5   avrcs          171865 non-null  float64
dtypes: float64(1), object(5)
memory usage: 8.2+ MB


```SAS
/* Step 2.2. Sample Selection, focus on Domestic Equity Mutual Funds */
/* for which the holdings data are the most complete and reliable */ 
data Equity_Funds; set Fund_Style;
if not missing (Lipper_Class) and lipper_class not in ('EIEI','G','LCCE','LCGE','LCVE','MCCE',
   'MCGE','MCVE','MLCE','MLGE','MLVE','SCCE','SCGE','SCVE') then delete;
else if missing(Lipper_Class) then do;
 if  not missing(si_obj_cd) and si_obj_cd not in ('AGG','GMC','GRI','GRO','ING','SCG') then delete;
 else if missing(si_obj_cd) then do;
  if not missing (wbrger_obj_cd) and wbrger_obj_cd not in ('G','G-I','AGG','GCI','GRI','GRO','LTG',
     'MCG','SCG') then delete;
  else if missing(wbrger_obj_cd) then do;
    if not missing(policy) and policy ne 'CS' then delete;
     else if not missing(policy) and avrcs<80 then delete;
end; end; end;
keep crsp_fundno;
run; 
```

In [51]:
# Replace the cells with '' as NAN
fund_style.replace('',np.nan,inplace = True)

# Remove the fund according to Lipper Class Code
Lipper_ClassIdxNA            = fund_style['lipper_class'].isna()
Lipper_ClassIdx              = fund_style['lipper_class'].isin(['EIEI','G','LCCE','LCGE','LCVE',
                                                                'MCCE','MCGE','MCVE','MLCE','MLGE',
                                                                'MLVE','SCCE','SCGE','SCVE'])
fund_style_Lipper_Class      = fund_style[Lipper_ClassIdx]
fund_style_Lipper_Class_Left = fund_style[Lipper_ClassIdxNA]
# fund_style_Lipper_Class.info()

# Remove the fund according to si_obj_cd
si_obj_cdIdxNA               = fund_style_Lipper_Class_Left['si_obj_cd'].isna()
si_obj_cdIdx                 = fund_style_Lipper_Class_Left['si_obj_cd'].isin(['AGG','GMC','GRI',
                                                                   'GRO','ING','SCG'])
fund_style_si_obj_cd         = fund_style_Lipper_Class_Left[si_obj_cdIdx]
fund_style_si_obj_cd_Left    = fund_style_Lipper_Class_Left[si_obj_cdIdxNA]

# Remove the funds according to wbrger_obj_cd
wbrger_obj_cdIdxNA           = fund_style_si_obj_cd_Left['wbrger_obj_cd'].isna()
wbrger_obj_cdIdx             = fund_style_si_obj_cd_Left['wbrger_obj_cd'].isin(['G','G-I','AGG','GCI',
                                                                         'GRI','GRO','LTG',
                                                                         'MCG','SCG'])
fund_style_wbrger_obj_cd      = fund_style_si_obj_cd_Left[wbrger_obj_cdIdx]
fund_style_wbrger_obj_cd_Left = fund_style_si_obj_cd_Left[wbrger_obj_cdIdxNA]
# fund_style_wbrger_obj_cd.info()

# Remove the fund according to 'policy'
policyIdx                     = fund_style_wbrger_obj_cd_Left['policy'].isin(['CS'])
fund_style_policy01           = fund_style_wbrger_obj_cd_Left[policyIdx]

policyIdxNA                   = fund_style_wbrger_obj_cd_Left['policy'].isna()
# Select those funds with more 80% invested in equity
avrcsIdx                      = fund_style_wbrger_obj_cd_Left['avrcs'] >= 80
fund_style_policy02           = fund_style_wbrger_obj_cd_Left[avrcsIdx | (policyIdxNA)]

Equity_Funds                  = pd.concat([fund_style_Lipper_Class,fund_style_si_obj_cd,
                                           fund_style_wbrger_obj_cd,fund_style_policy01,
                                           fund_style_policy02])
del fund_style_Lipper_Class_Left,fund_style_si_obj_cd_Left,fund_style_wbrger_obj_cd_Left,
#Equity_Funds.info()
Equity_Funds = Equity_Funds[['crsp_fundno']]
Equity_Funds = Equity_Funds.sort_values('crsp_fundno').reset_index(drop = True)
Equity_Funds.head()

,crsp_fundno
0,000001
1,000002
2,000002
3,000002
4,000003


In [52]:
Equity_Funds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69542 entries, 0 to 69541
Data columns (total 1 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   crsp_fundno  69542 non-null  object
dtypes: object(1)
memory usage: 543.4+ KB


```SAS
/* Step 2.3. Get CRSP Mutual Fund, Monthly Net Returns */
/* Net Returns include distributions (dividends and capital gains) */
/* and are net of total expenses (CRSP MFDB manual, page 6) */
proc sql;
 create table MF_TotRet
 as select a.date format date9.,a.crsp_fundno,a.mret,b.mtna
 from (select intnx("month",caldt,0,"e") as Date,crsp_fundno,mret
        from Crsp.Monthly_Returns where not missing(mret)) a,
      (select intnx("month",caldt,0,"e") as Date,crsp_fundno,mtna
          from Crsp.Monthly_Tna) b
      where a.date=b.date and a.crsp_fundno=b.crsp_fundno
      and a.crsp_fundno in (select distinct crsp_fundno from equity_funds)
      and a.date >= "&begdate"d;
quit; 

```

In [53]:
crsp_monthly_returns,meta     = pyreadstat.read_sas7bdat('crsp_monthly_returns.sas7bdat',
                                                         usecols = ['crsp_fundno', 'caldt',
                                                                    'mret'])

In [54]:
# Rename the 'caldt' as 'Date'
crsp_monthly_returns.rename(columns = {'caldt':'date'},inplace = True)
crsp_monthly_returns['date'] = pd.to_timedelta(crsp_monthly_returns['date'], 
                                               unit='d') + pd.Timestamp(1960, 1, 1)

In [55]:
# The section of code get the CRSP Mutual Fund net return and total assets
# - Net Returns include distributions (dividends and capital gains) and are net of total
#   expenses (CRSP MFDB manual, page 6)

# Align the elements in 'Date' to the end of each month
crsp_monthly_returns['date'] = crsp_monthly_returns['date'] + MonthEnd(0)
# Transform the 'crsp_fundno' into string with 6 characters
crsp_monthly_returns['crsp_fundno'] = crsp_monthly_returns['crsp_fundno'].astype(int)
crsp_monthly_returns['crsp_fundno'] = crsp_monthly_returns['crsp_fundno'].astype(str)
crsp_monthly_returns['crsp_fundno'] = crsp_monthly_returns['crsp_fundno'].str.zfill(6)

# Remove the NAN terms in columns 'mret'
# crsp_monthly_returns = crsp_monthly_returns.loc[crsp_monthly_returns['mret'].dropna().index]
# crsp_monthly_returns = crsp_monthly_returns[crsp_monthly_returns['mret'].notnull()]
crsp_monthly_returns = crsp_monthly_returns.dropna(subset = ['mret']).reset_index(drop = True)

In [56]:
crsp_monthly_tna,meta     = pyreadstat.read_sas7bdat('crsp_monthly_tna.sas7bdat')

In [57]:
crsp_monthly_tna.rename(columns = {'caldt':'date'},inplace = True)
crsp_monthly_tna['date']  = pd.to_timedelta(crsp_monthly_tna['date'], 
                                            unit='d') + pd.Timestamp(1960, 1, 1)

In [58]:
# Align the elements in 'Date' to the end of each month
crsp_monthly_tna['date']        = crsp_monthly_tna['date'] + MonthEnd(0)
# Transform the 'crsp_fundno' into string with 6 characters
crsp_monthly_tna['crsp_fundno'] = crsp_monthly_tna['crsp_fundno'].astype(int)
crsp_monthly_tna['crsp_fundno'] = crsp_monthly_tna['crsp_fundno'].astype(str)
crsp_monthly_tna['crsp_fundno'] = crsp_monthly_tna['crsp_fundno'].str.zfill(6)

In [59]:
crsp_monthly_returns = crsp_monthly_returns[crsp_monthly_returns['crsp_fundno'].isin(Equity_Funds['crsp_fundno'].unique())]
crsp_monthly_returns = crsp_monthly_returns[crsp_monthly_returns['date'] >= begdate]

In [60]:
MF_TotRet            = pd.merge(crsp_monthly_returns,crsp_monthly_tna,on = ['crsp_fundno','date'])

In [61]:
MF_TotRet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4204789 entries, 0 to 4204788
Data columns (total 4 columns):
 #   Column       Dtype         
---  ------       -----         
 0   crsp_fundno  object        
 1   date         datetime64[ns]
 2   mret         float64       
 3   mtna         float64       
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 128.3+ MB


```SAS

/* Extracting and linking expense ratio data required to get Raw Return */
proc sql;
create table MF_TotRet
     as select a.*, b.exp_ratio
     from MF_TotRet as a left join Crsp.Fund_Fees as b
     on a.crsp_fundno=b.crsp_fundno and a.date between b.begdt and b.enddt
     order by crsp_fundno,date;
quit; 
```

In [62]:
crsp_fund_fees,meta    = pyreadstat.read_sas7bdat('crsp_fund_fees.sas7bdat',
                                                  usecols = ['CRSP_FundNo', 'begdt',
                                                             'enddt','exp_ratio'])
# Transform the columns 'begdt' and 'enddt' into usual datetime
crsp_fund_fees['begdt'] = pd.to_timedelta(crsp_fund_fees['begdt'], 
                                          unit='d') + pd.Timestamp(1960, 1, 1)
crsp_fund_fees['enddt'] = pd.to_timedelta(crsp_fund_fees['enddt'], 
                                          unit='d') + pd.Timestamp(1960, 1, 1)

In [63]:
crsp_fund_fees['CRSP_FundNo'] = crsp_fund_fees['CRSP_FundNo'].astype(int)
crsp_fund_fees['CRSP_FundNo'] = crsp_fund_fees['CRSP_FundNo'].astype(str)
crsp_fund_fees['CRSP_FundNo'] = crsp_fund_fees['CRSP_FundNo'].str.zfill(6)

In [64]:
MF_TotRet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4204789 entries, 0 to 4204788
Data columns (total 4 columns):
 #   Column       Dtype         
---  ------       -----         
 0   crsp_fundno  object        
 1   date         datetime64[ns]
 2   mret         float64       
 3   mtna         float64       
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 128.3+ MB


In [65]:
crsp_fund_fees.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 524921 entries, 0 to 524920
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   CRSP_FundNo  524921 non-null  object        
 1   begdt        524921 non-null  datetime64[ns]
 2   enddt        524921 non-null  datetime64[ns]
 3   exp_ratio    524803 non-null  float64       
dtypes: datetime64[ns](2), float64(1), object(1)
memory usage: 16.0+ MB


In [ ]:
# Memory-efficient approach: Process in chunks
# Only keep necessary columns from MF_TotRet
MF_TotRet_subset = MF_TotRet[['crsp_fundno', 'date']].copy()

# Get unique funds from MF_TotRet
unique_funds = MF_TotRet_subset['crsp_fundno'].unique()
print(f"Processing {len(unique_funds)} unique funds...")

# Rename fees column
crsp_fund_fees_renamed = crsp_fund_fees.rename(columns={'CRSP_FundNo': 'crsp_fundno'})

# Pre-filter fees to relevant date range
min_date = MF_TotRet_subset['date'].min()
max_date = MF_TotRet_subset['date'].max()
fees_filtered = crsp_fund_fees_renamed[
    crsp_fund_fees_renamed['crsp_fundno'].isin(unique_funds) &
    (crsp_fund_fees_renamed['begdt'] <= max_date) & 
    (crsp_fund_fees_renamed['enddt'] >= min_date)
][['crsp_fundno', 'begdt', 'enddt', 'exp_ratio']].copy()

del crsp_fund_fees_renamed

# Function to find matching fee for each row
def find_fee(group):
    # For each date in group, find the fee where begdt <= date <= enddt
    dates = group['date'].values
    fees = group[['crsp_fundno']].copy()
    fees['exp_ratio'] = np.nan
    
    # Get fees for this fund
    fund_fees = fees_filtered[fees_filtered['crsp_fundno'] == fees['crsp_fundno'].iloc[0]]
    
    if len(fund_fees) == 0:
        return fees['exp_ratio']
    
    # For each date, find matching fee
    result = []
    for d in dates:
        match = fund_fees[(fund_fees['begdt'] <= d) & (fund_fees['enddt'] >= d)]
        if len(match) > 0:
            result.append(match['exp_ratio'].iloc[0])
        else:
            result.append(np.nan)
    return pd.Series(result, index=fees.index)

# Process in chunks by fund
print("Processing fees by fund...")
chunks = []
chunk_size = 5000
funds_list = list(unique_funds)

for i in range(0, len(funds_list), chunk_size):
    chunk_funds = funds_list[i:i+chunk_size]
    chunk_data = MF_TotRet_subset[MF_TotRet_subset['crsp_fundno'].isin(chunk_funds)].copy()
    
    # For each fund, apply fee matching
    fees_results = []
    for fund in chunk_funds:
        fund_data = chunk_data[chunk_data['crsp_fundno'] == fund].copy()
        fund_fees = fees_filtered[fees_filtered['crsp_fundno'] == fund]
        
        if len(fund_fees) > 0:
            # Vectorized approach for this fund
            for idx in fund_data.index:
                d = fund_data.loc[idx, 'date']
                match = fund_fees[(fund_fees['begdt'] <= d) & (fund_fees['enddt'] >= d)]
                if len(match) > 0:
                    fund_data.loc[idx, 'exp_ratio'] = match['exp_ratio'].iloc[0]
        
        fees_results.append(fund_data[['exp_ratio']])
    
    chunk_exp_ratio = pd.concat(fees_results)
    chunk_data['exp_ratio'] = chunk_exp_ratio['exp_ratio']
    chunks.append(chunk_data)
    
    if i % 10000 == 0:
        print(f"Processed {i}/{len(funds_list)} funds...")

MF_TotRet02 = pd.concat(chunks, ignore_index=True)
MF_TotRet02 = MF_TotRet02.merge(MF_TotRet, on=['crsp_fundno', 'date'], how='left')

del chunks, fees_filtered, MF_TotRet_subset
MF_TotRet02.info()

In [70]:
# Step 1: Rename column for merge
crsp_fund_fees_renamed = crsp_fund_fees.rename(columns={'CRSP_FundNo': 'crsp_fundno'})

# Step 2: Get date range from MF_TotRet to pre-filter fees (reduces rows before join)
min_date = MF_TotRet['date'].min()
max_date = MF_TotRet['date'].max()

# Pre-filter fees data to reduce memory usage
fees_filtered = crsp_fund_fees_renamed[
    (crsp_fund_fees_renamed['begdt'] <= max_date) & 
    (crsp_fund_fees_renamed['enddt'] >= min_date)
]

# Step 3: Perform left merge
MF_TotRet02 = pd.merge(MF_TotRet, fees_filtered, on='crsp_fundno', how='left')

# Step 4: Apply date filter (keeps only matching date ranges)
MF_TotRet02 = MF_TotRet02[
    (MF_TotRet02['date'] >= MF_TotRet02['begdt']) & 
    (MF_TotRet02['date'] <= MF_TotRet02['enddt']) |
    MF_TotRet02['exp_ratio'].isna()
]

del fees_filtered, crsp_fund_fees_renamed
MF_TotRet02.info()

MemoryError: Unable to allocate 531. MiB for an array with shape (69566006,) and data type int64

In [ ]:
MF_TotRet04 = MF_TotRet02[((MF_TotRet02['date'] >= MF_TotRet02['begdt']) & 
                          (MF_TotRet02['date'] <= MF_TotRet02['enddt'])) |
                          MF_TotRet02['exp_ratio'].isna()]
MF_TotRet04.info()

In [ ]:
MF_TotRet04 = MF_TotRet04.drop(['begdt','enddt'],axis = 1).reset_index(drop = True).sort_values(['crsp_fundno','date'])
MF_TotRet04.info()

In [ ]:
MF_TotRet04.describe()

In [ ]:
del MF_TotRet02

```SAS
/* Step 2.4. Get Raw Total Portfolio Return by adding back expenses */
data MF_TotRet; set MF_TotRet;
by crsp_fundno date; retain TNA;
if exp_ratio = -99 then exp_ratio=.;
* Populate assets forward for gaps in asset reporting ;
if first.crsp_fundno or MTNA>0 then TNA=MTNA;
* Weight of Share Class in the Mutual Fund Portfolio;
PORTW = lag(TNA); if first.crsp_fundno then PORTW=TNA;
/* if missing weight, then assume $1million asset weight */
if missing(PORTW) then PORTW=1;
RRET=sum(mret,exp_ratio/12);
label MRET = "Monthly Returns, Net" RRET = "Monthly Returns, Raw";
label PORTW= "Mutual Fund Share Class Portfolio Weight, Lagged TNA";
format mret rret exp_ratio percentn8.4;	drop MTNA;
run; 
```

In [ ]:
MF_TotRet04,meta = pyreadstat.read_sas7bdat('mf_totret.sas7bdat')

In [ ]:
MF_TotRet04.describe() # This is the same as the one obtained from SAS code

In [ ]:
# Replace the -99 in 'exp_ratio' columns as NaN
MF_TotRet04['exp_ratio'][MF_TotRet04['exp_ratio'] == -99] = np.nan

# Construct a new column 'tna', Populate assets forward for gaps in asset reporting
# Weight of Share Class in the Mutual Fund Portfolio;
MF_TotRet04['tna'] = np.nan
# assign the values when 'mtna' > 0 or the first row of every 'crsp_fundno'
MF_TotRet04['tna'][MF_TotRet04['mtna'] > 0] = MF_TotRet04['mtna'][MF_TotRet04['mtna'] > 0]
Tmp                = MF_TotRet04.reset_index().groupby('crsp_fundno').first()
Tmp['tna']         = Tmp['mtna']
MF_TotRet04.loc[Tmp['index'],'tna']         = Tmp['tna'].values
# Remember the 'RETAIN' command in SAS, which is same as fillna method as follows
MF_TotRet04['tna'] = MF_TotRet04['tna'].ffill()

In [ ]:
# Construct the Weight of Share Class in the Mutual Fund Portfolio, 'portw'
MF_TotRet04['portw'] = MF_TotRet04.groupby('crsp_fundno')['tna'].shift(1).values
Tmp                  = MF_TotRet04.reset_index().groupby('crsp_fundno').first()
Tmp['portw']         = Tmp['tna']
MF_TotRet04.loc[Tmp['index'],'portw']             = Tmp['portw'].values
# if missing weight, then assume $1million asset weight
MF_TotRet04['portw'][MF_TotRet04['portw'].isna()] = 1

In [ ]:
# MRET = "Monthly Returns, Net"
# RRET = "Monthly Returns, Raw"
# PORTW= "Mutual Fund Share Class Portfolio Weight, Lagged TNA"
# In SAS, the Sum(a,b) will not remove the nan terms, hence here we need to fill the 
# na terms with 0 
MF_TotRet04['rret']= MF_TotRet04['mret'] + MF_TotRet04['exp_ratio'].fillna(0)/12

MF_TotRet04.drop('mtna',inplace = True,axis = 1)

In [ ]:
MF_TotRet = MF_TotRet04.copy()
del MF_TotRet04

In [ ]:
MF_TotRet['crsp_fundno'] = MF_TotRet['crsp_fundno'].astype(int)
MF_TotRet['crsp_fundno'] = MF_TotRet['crsp_fundno'].astype(str)
MF_TotRet['crsp_fundno'] = MF_TotRet['crsp_fundno'].str.zfill(6)
MF_TotRet.info()

In [ ]:
MF_TotRet.describe()

In [ ]:
MF_TotRet.head()

```SAS
/* Step 2.5. Merge with Portfolio Identifier from MFLINKS */
proc sql;
create table MF_TotRet 
as select b.wficn, a.*
from MF_TotRet as a, mfl.mflink1 as b
where a.crsp_fundno=b.crsp_fundno and not missing(b.wficn)
order by wficn, date, crsp_fundno;
quit; 
```

In [ ]:
mfl_mflink1,meta = pyreadstat.read_sas7bdat('mfl_mflink1.sas7bdat')

In [ ]:
mfl_mflink1['crsp_fundno'] = mfl_mflink1['crsp_fundno'].astype(int)
mfl_mflink1['crsp_fundno'] = mfl_mflink1['crsp_fundno'].astype(str)
mfl_mflink1['crsp_fundno'] = mfl_mflink1['crsp_fundno'].str.zfill(6)
mfl_mflink1['wficn']       = mfl_mflink1['wficn'].astype(int)
mfl_mflink1.head()

In [ ]:
MF_TotRet02 = pd.merge(MF_TotRet,mfl_mflink1,on = ['crsp_fundno'])
MF_TotRet02.describe()

```SAS
/* Step 2.6. Aggregate multiple share classes, use lagged TNA as weights */
/* Compute monthly net and raw returns, TNA and expense ratio at the portfolio level */
/* level for fund portfolios with multiple share classes */
proc sql; 
create table MF_Port
as select wficn, date,
    sum(exp_ratio*portw)/sum(portw) as exp_ratio format percentn8.4,
    sum(tna) as mtna format dollar12.3, 
    count(crsp_fundno) as nclass label='# of Share Classes', 
	sum(mret*portw)/sum(portw) as mret format percentn8.4,
    sum(rret*portw)/sum(portw) as rret format percentn8.4
from MF_TotRet
group by wficn, date;
quit; 
```

In [ ]:
MF_PortGB = MF_TotRet02.copy()
MF_PortGB['mret*portw']  = MF_PortGB['mret'] * MF_PortGB['portw']
MF_PortGB['rret*portw']  = MF_PortGB['rret'] * MF_PortGB['portw']
MF_PortGB['tna*portw']  = MF_PortGB['tna'] * MF_PortGB['portw']
MF_PortGB['exp_ratio*portw']   = MF_PortGB['exp_ratio'] * MF_PortGB['portw']
MF_PortGB['missing_exp_ratio'] = MF_PortGB['exp_ratio'].isna()

In [ ]:
MF_PortGB = MF_PortGB.groupby(['wficn','date'])[['exp_ratio','crsp_fundno','portw',
                                                'exp_ratio*portw','tna','rret*portw','tna*portw',
                                                'mret*portw','missing_exp_ratio']]

In [ ]:
pd.Series([np.nan,5]).sum(skipna = False)

In [ ]:
MF_Port                = pd.DataFrame()
MF_Port['nclass']      = MF_PortGB.count()['crsp_fundno']
MF_Port['tna']         = MF_PortGB.sum()['tna']
MF_Port['missing_exp_ratio'] = MF_PortGB.sum()['missing_exp_ratio']
MF_Port['tna2']         = MF_PortGB.sum()['tna*portw']/MF_PortGB.sum()['portw']
MF_Port['exp_ratio']   = MF_PortGB.sum()['exp_ratio*portw']/MF_PortGB.sum()['portw']
MF_Port['rret']        = MF_PortGB.sum()['rret*portw']/MF_PortGB.sum()['portw']
MF_Port['mret']        = MF_PortGB.sum()['mret*portw']/MF_PortGB.sum()['portw']

In [ ]:
Tmp =  MF_PortGB.sum()['portw']
(Tmp == 0).sum()

In [ ]:
MF_PortGB.sum()['exp_ratio*portw'].plot()

In [ ]:
MF_Port['exp_ratio'].plot()

In [ ]:
(MF_Port['missing_exp_ratio'] == 0).sum()

In [ ]:
(MF_PortGB.sum()['portw'] == 0).sum()

In [ ]:
MF_Port                = MF_Port.reset_index()

In [ ]:
MF_Port['exp_ratio'][MF_Port['missing_exp_ratio'] != 0] = np.nan

In [ ]:
# MF_PortGB['portw'] is correct
# there are some observations MF_PortGB.sum()['portw'] == 0, which result in missing values
MF_Port.describe()

```SAS
* Step 3.1. Combining Hypothetical Holdings Returns with CRSP Total Returns */
data RetGap;
merge MF_PortRets(in=a) MF_Port(in=b);
by wficn date; if a and b; 
RetGap = RRET - HRET; 
format RETGAP MRET RRET HRET percentn8.4;
label RetGap="Return Gap";
run;

/* Calculate average return gap over previous 12 months */
proc expand data=RetGap out=RetGap;
by wficn; id date;
convert retgap=retgap12m /transformout=(nomiss movave 12 trimleft 11);
quit; 

/* Holdings are reported to the SEC and captured by Thomson with 3-month delay */
data RetGap; set RetGap; by wficn date; format retgap12m_4 percentn8.4;
retgap12m_4=lag4(retgap12m);
if first.wficn then retgap12m_4=.;
run; 
```

```SAS
/* Form Return Gap Portfolios based on deciles of average 12-month Return Gap */
/* Number of portfolios to form based on Return Gap Measure */
%let portn=10; 
proc sort data=RetGap; by date;run;
proc rank data=Retgap out=RetGapRank groups=&portn;
by date; var retgap12m_4;
ranks RetGap_Port;
data RetGapRank;set RetGapRank; RetGap_Port=RetGap_Port+1;
proc sort data=RetGapRank; by date RetGap_Port;run; 

/* Equal-weighted Return Gap Portfolios*/
proc means data=RetGapRank noprint;
where not missing(RetGap_Port);
by date RetGap_Port;
output out=RetGapPort mean(mret)=Port_Ret;
run;
proc transpose data=RetGapPort out=RetGapPort (drop=_NAME_) prefix=port_;
by date; id RetGap_Port; var port_ret; 
run; 
```

```SAS
/* Merge in Risk Factors, KSZ (2008) use CAPM, FF and Carhart risk models */
proc sql;
 create table RetGapPort
      as select a.*, b.mktrf, b.smb, b.hml, b.umd, b.rf
      from RetGapPort a left join ff.factors_monthly b
      on a.date=intnx('month',b.dateff,0,'E');
quit; 

/* Replicate Panel A of Figure 2 of (KSZ (2008), page 2396 */
/* Compute abnormal returns according to different risk models */
%macro Trading_Strategies;
data Reg; set RetGapPort;
where date between intnx('year',"&begdate"d,4,'b') and "&enddate"d;
%do k=1 %to &portn;
 exret_&k=port_&k-rf;
 abret_&k=port_&k-rf-mktrf;
%end;
run; options nonotes;
%do k=1 %to &portn;
 proc reg data=Reg outest=m1 (keep=intercept) noprint; model port_&k=; 
 proc reg data=Reg outest=m2 (keep=intercept) noprint; model abret_&k=; 
 proc reg data=Reg outest=m3 (keep=intercept) noprint; model exret_&k=mktrf; 
 proc reg data=Reg outest=m4 (keep=intercept) noprint; model exret_&k=mktrf smb hml; 
 proc reg data=Reg outest=m5 (keep=intercept) noprint; model exret_&k=mktrf smb hml umd; 
run; quit; 
proc append base=MeanRaw   data=m1 force;
proc append base=MeanAbret data=m2 force;
proc append base=MeanCAPM  data=m3 force;
proc append base=MeanFF    data=m4 force;
proc append base=MeanFFM   data=m5 force;run;
proc sql; drop table m1,m2,m3,m4,m5; quit; 
%end; options notes; %mend; 
%Trading_Strategies; 

/* Combine abnormal returns across different risk-adjustment models */
data PlotRets; format RetGap_Port 3.; merge
 Meanraw (rename=(intercept=meanraw))
 Meanabret (rename=(intercept=meanabret))
 MeanCAPM (rename=(intercept=meanCAPM))
 MeanFF (rename=(intercept=meanFF))
 MeanFFM (rename=(intercept=meanFFM));
RetGap_Port +1;
format meanraw meanabret meanCAPM meanFF meanFFM percentn8.2;
label meanraw='Raw Return' meanabret='Excess Return' meanCAPM='CAPM Alpha'
meanFF='Fama-French Alpha' meanFFM='Carhart Alpha' RetGap_Port='Return Gap Portfolio';
run; 
proc datasets library=work nolist; delete mean: holdings: ret mf_:; run; quit;

/* Plot Abnormal returns across Return Gap-based portfolios */
options nodate orientation=landscape; ods pdf file='Return_Gap_Plot.pdf';
goptions device=pdfc; /* Plot Saved in Home Director */
%let vars=meanabret meanCAPM meanFF meanFFM;
axis1 label=(angle=90 "Abnormal Return Of Trading Strategy");
axis2 label=("Return Gap Portfolio");
symbol interpol=join w=3 l=1;
proc gplot data =PlotRets;
 Title "Abnormal Return of Return Gap Trading Strategy";
 Title2 "Sorting Based on the Return Gap ";
 plot (&vars)*RetGap_Port / overlay legend vaxis=axis1 haxis=axis2;
run;quit; ods pdf close;
```